In [1]:
2+2

4

In [2]:
with open("zolo.html", "r", encoding='utf-8') as f:
    text= f.read()

In [44]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim


class extract_properties:
    
    def __init__(self):
        pass
        
    columns = ['unit_num', 'street', 'city', 'prov', 
               'district', 'latitude', 'longitude', 
               'sold_date', 'dom', 'sold_price',
               'listing_price', 'beds', 'baths', 
               'size', 'mls', 'brokerage']

        
    def to_dataframe(self, zolo_text, columns = columns):
        zolo_prop = zolo_text.split("data-propertyid")
        
        prop_data = pd.DataFrame(columns=columns)
        
        for prop in zolo_prop:
            if "MLS" in prop:
                row = {}
                details = prop.split("&")

                unit_num = details[13].split(';')[1]

                street = details[15].split(';')[1]

                city = details[19].split(';')[1]

                prov = details[23].split(';')[1]

                district = details[35].split(';')[1]

                #latitude
                latitude = details[42].split('=')[1].split()[0][1:-1]


                #longitude
                longitude = details[44].split('=')[1].split()[0][1:-1]

                # sold date
                sold_date = details[51].split('•')[1]

                # days on market
                dom = details[55].split('"p1">')[1]

                # sold price
                sold_price = details[63].split('$')[1]
                
                # listing price
                listing_price = details[67].split('$')[1]

                # num of beds
                beds = details[75].split(';')[1]

                # num of baths
                baths = details[79].split(';')[1]

                # size info may not be available, subsequent detail index will change
                # check if size detail exists in listing, 600-699 sqft or 1500-2000 sqft
                if ('99 sqft' in prop) or ('00 sqft' in prop):
                    size = details[83].split(';')[1]
                    mls = details[91].split(';')[1]
                    brokerage = details[99].split(';')[1]

                else:
                    size = np.NaN
                    mls = details[87].split(';')[1]
                    brokerage = details[95].split(';')[1]


                row = [unit_num, street, city, prov, district, latitude, longitude, 
                       sold_date, dom, sold_price, listing_price, beds, baths, size, mls, brokerage]

                row = pd.DataFrame([row], columns = columns)

                prop_data = pd.concat([prop_data, row])
                
        
        # remove all unwanted spaces in each columns
        def strip_string(value):
            # Check if the value is NaN (None) and return it as is
            if pd.isna(value):
                return value
            # Otherwise, apply the strip() method to remove leading and trailing whitespaces
            return value.strip()
        
        prop_data = prop_data.applymap(strip_string)   

        property_df = prop_data.reset_index(drop=True)
        
        return property_df
    
    def clean_beds(self, property_df):
        # remove the bd suffixes
        property_df['beds'] = property_df['beds'].apply(lambda x: x.split()[0])
       
    
    def clean_baths(self, property_df):
        # remove the ba suffixes
        property_df['baths'] = property_df['baths'].apply(lambda x: x.split()[0])
        
    def clean_dom(self, property_df):
        property_df['dom'] = property_df['dom'].apply(lambda x: x.split()[0])
        property_df['dom'] = pd.to_numeric(property_df['dom'])
        
    def clean_sold_date(self, property_df):
        def get_sold_date(date, currentYear=2023):
            months_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 
                           'Jun', 'Jul', 'Aug', 'Sep', 'Oct']
            month = date.split()[0]
            month_num = months_list.index(month)

            date_num = date.split()[1].split('th')[0]
            return pd.to_datetime(f'{currentYear}-{month_num+1}-{date_num}')

        property_df['sold_date'] = property_df['sold_date'].apply(lambda x: get_sold_date(x))

    def clean_prices(self, property_df):
        
        """
        Remvoe the comma and change price column datatypes to numeric
        """
        def remove_comma(x):
            """
            Remove the comma in he prices
            Input: '625,000'
            Ouput: '625000'
            """
            number = ''
            for i in x.split(','):
                number += i
            return number
        
        property_df[['sold_price','listing_price']] = property_df[['sold_price','listing_price']].applymap(remove_comma)
            
        property_df[['sold_price','listing_price']] = property_df[['sold_price','listing_price']].applymap(pd.to_numeric)
    
            
    ## make function, add_price_Difference
    def add_listing_date(self, property_df):
        # get listing date
        listing_date = property_df['sold_date'] - pd.to_timedelta(property_df['dom'],
                                                                  unit='days')
        # add to main dataset
        property_df['listing_date'] = listing_date
        
    def estimate_size(self, property_df):
        # clean unit sizes
        size_range = property_df['size'].apply( lambda x: x.split()[0] if type(x) == str else x)

        # get average size of range
        size = size_range.apply(lambda x: ( int(x.split('-')[0]) + int(x.split('-')[1]) ) // 2 if type(x) == str else x )

        # round off size to end in 00.0, ie. 500
        size = size.apply(lambda x: x + 1 if x%2 == 1 else x)
        size = size.astype('Int64')

        property_df['size'] = size
        
    def add_street_num(self, property_df):
        # get the street number
        property_df['street_num'] = property_df['unit_num'].apply(lambda x: x.split('-')[-1])

    def add_app_num(self, property_df):
        # if listing is not an apartment give the appartment number -999
        property_df['app_num'] = property_df['unit_num'].apply(lambda x: x.split('-')[0] if len(x.split('-'))>1 else -999)

    def add_full_address(self, property_df):
        # get lat and long coordinates from full address
        property_df['full_address'] = property_df[['street_num','street','city','prov']].apply(lambda row: ' '.join(row), axis=1)

        
    def get_coordinates(self, address):
            """
            Helper function for function check_geo_coordinates to get latitude and longitude coordinates 
            from the library geopy.
            Input the address and return the latitude and longitude.
            """
            # Initialize a geocoder with the Nominatim service
            geolocator = Nominatim(user_agent="forward_geocoder")

            # Use the geocoder to get the location details (longitude and latitude)
            location = geolocator.geocode(address)

            if location:
                latitude = location.latitude
                longitude = location.longitude
                return latitude, longitude
            
        
    def check_geo_coordinates(self, property_df):
        """
        This function compares the zolo coordinates and geopy coorindates.
        If the sets of coordinates differ by 0.1 degree in either direction, 
            then return those rows for further review.
            
        0.001 degree is approximately 0.111km = 111 meters
        """
        
        # call on another class function to get coorindates
        geo_coords = pd.DataFrame(tuple(property_df['full_address'].apply(self.get_coordinates )), columns=['lat','long'])
            
        # update latitude and longitude data type
        property_df[['latitude','longitude']] = property_df[['latitude','longitude']].apply(pd.to_numeric)

        # compare the coordinates
        coords_df =  pd.DataFrame(columns =['lat_diff', 'long_diff'])

        coords_df['lat_diff'] = np.abs(property_df['latitude'] - geo_coords['lat'])
        coords_df['long_diff'] = np.abs(property_df['longitude'] - geo_coords['long'])

        # 0.1 diff in lat is 11.1km 
        rows_to_check = coords_df[(coords_df['lat_diff'] > 0.001) | (coords_df['long_diff'] > 0.1)].index
        return property_df.iloc[rows_to_check]

In [45]:
apple = extract_properties()
raw_df = apple.to_dataframe(text)

raw_df

,unit_num,street,city,prov,district,latitude,longitude,sold_date,dom,sold_price,listing_price,beds,baths,size,mls,brokerage
0,1101-155,Yorkville Avenue,Toronto,ON,Annex,43.6704,-79.3943,Jul 28th,18 days on market,"640,000","645,000",1 bd,1 ba,500-599 sqft,C6653090,RIGHT AT HOME REALTY
1,57-847,Sheppard Avenue,Toronto,ON,Clanton Park,0,0,Jul 28th,99 days on market,"870,000","899,900",2 bd,2 ba,1000-1199 sqft,C6039759,"RE/MAX PRIME PROPERTIES - UNIQUE GROUP, BROKERAGE"
2,808-66,Forest Manor Road,Toronto,ON,Henry Farm,43.7744,-79.3459,Jul 28th,10 days on market,"828,500","788,000",2 bd,2 ba,800-899 sqft,C6674154,BAY STREET GROUP INC.
3,67-1359,Neilson Road,Toronto,ON,Malvern,43.8089,-79.2194,Jul 28th,42 days on market,"700,000","569,000",3 bd,3 ba,1000-1199 sqft,E6168144,RE/MAX ROYAL PROPERTIES REALTY
4,224-9,Mabelle Avenue,Toronto,ON,Islington-city Centre West,43.6461,-79.526,Jul 28th,11 days on market,"495,000","499,900",1 bd,1 ba,500-599 sqft,W6674724,ROYAL LEPAGE SIGNATURE REALTY
5,910-761,Bay Street,Toronto,ON,Bay Street Corridor,43.6597,-79.385,Jul 28th,14 days on market,"803,000","818,000",1 bd,2 ba,700-799 sqft,C6666920,ROYAL LEPAGE CONNECT REALTY
6,96,Roseheath Avenue,Toronto,ON,East End-danforth,43.6835,-79.318,Jul 28th,9 days on market,"1,401,000","999,900",3 bd,3 ba,NaN,E6676322,ROYAL LEPAGE GOLDEN RIDGE REALTY
7,202-20,Marina Avenue,Toronto,ON,Long Branch,43.5937,-79.5328,Jul 28th,37 days on market,"8,200,000","888,888",2 bd,2 ba,1000-1199 sqft,W6190872,RIGHT AT HOME REALTY
8,11-38,Gibson Avenue,Toronto,ON,Weston,43.7017,-79.5072,Jul 28th,66 days on market,"625,000","629,000",2 bd,2 ba,900-999 sqft,W6029476,REAL ONE REALTY INC.
9,621-55,Merchant's Wharf,Toronto,ON,Waterfront Communities C8,43.6447,-79.3641,Jul 28th,22 days on market,"738,000","759,000",1 bd,1 ba,600-699 sqft,C6641856,RE/MAX PROFESSIONALS INC.


In [46]:
apple.clean_beds(raw_df)
apple.clean_baths(raw_df)
apple.clean_dom(raw_df)
apple.clean_sold_date(raw_df)
apple.clean_prices(raw_df)
apple.add_listing_date(raw_df)
apple.estimate_size(raw_df)
apple.add_street_num(raw_df)
apple.add_app_num(raw_df)
apple.add_full_address(raw_df)

In [47]:
raw_df.shape

(24, 20)

In [48]:
raw_df.head()

,unit_num,street,city,prov,district,latitude,longitude,sold_date,dom,sold_price,listing_price,beds,baths,size,mls,brokerage,listing_date,street_num,app_num,full_address
0,1101-155,Yorkville Avenue,Toronto,ON,Annex,43.6704,-79.3943,2023-07-28,18,640000,645000,1,1,550,C6653090,RIGHT AT HOME REALTY,2023-07-10,155,1101,155 Yorkville Avenue Toronto ON
1,57-847,Sheppard Avenue,Toronto,ON,Clanton Park,0,0,2023-07-28,99,870000,899900,2,2,1100,C6039759,"RE/MAX PRIME PROPERTIES - UNIQUE GROUP, BROKERAGE",2023-04-20,847,57,847 Sheppard Avenue Toronto ON
2,808-66,Forest Manor Road,Toronto,ON,Henry Farm,43.7744,-79.3459,2023-07-28,10,828500,788000,2,2,850,C6674154,BAY STREET GROUP INC.,2023-07-18,66,808,66 Forest Manor Road Toronto ON
3,67-1359,Neilson Road,Toronto,ON,Malvern,43.8089,-79.2194,2023-07-28,42,700000,569000,3,3,1100,E6168144,RE/MAX ROYAL PROPERTIES REALTY,2023-06-16,1359,67,1359 Neilson Road Toronto ON
4,224-9,Mabelle Avenue,Toronto,ON,Islington-city Centre West,43.6461,-79.526,2023-07-28,11,495000,499900,1,1,550,W6674724,ROYAL LEPAGE SIGNATURE REALTY,2023-07-17,9,224,9 Mabelle Avenue Toronto ON


In [49]:
apple.check_geo_coordinates(raw_df)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=155+Yorkville+Avenue+Toronto+ON&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [ ]:
correct_coords_1 = apple.get_coordinates('847 Sheppard Avenue W Toronto ON')
correct_coords_2 = apple.get_coordinates('66 Forest Manor Road Toronto ON')

In [9]:
raw_df.loc[1, ['latitude','longitude']]=correct_coords_1
raw_df.loc[1, ['latitude','longitude']]=correct_coords_2
raw_df

NameError: name 'correct_coords_1' is not defined

In [21]:
raw_df

,unit_num,street,city,prov,district,latitude,longitude,sold_date,dom,sold_price,listing_price,beds,baths,size,mls,brokerage,listing_date,street_num,app_num,full_address
0,1101-155,Yorkville Avenue,Toronto,ON,Annex,43.6704,-79.3943,2023-07-28,18,"640,000","645,000",1,1,550,C6653090,RIGHT AT HOME REALTY,2023-07-10,155,1101,155 Yorkville Avenue Toronto ON
1,57-847,Sheppard Avenue,Toronto,ON,Clanton Park,0,0,2023-07-28,99,"870,000","899,900",2,2,1100,C6039759,"RE/MAX PRIME PROPERTIES - UNIQUE GROUP, BROKERAGE",2023-04-20,847,57,847 Sheppard Avenue Toronto ON
2,808-66,Forest Manor Road,Toronto,ON,Henry Farm,43.7744,-79.3459,2023-07-28,10,"828,500","788,000",2,2,850,C6674154,BAY STREET GROUP INC.,2023-07-18,66,808,66 Forest Manor Road Toronto ON
3,67-1359,Neilson Road,Toronto,ON,Malvern,43.8089,-79.2194,2023-07-28,42,"700,000","569,000",3,3,1100,E6168144,RE/MAX ROYAL PROPERTIES REALTY,2023-06-16,1359,67,1359 Neilson Road Toronto ON
4,224-9,Mabelle Avenue,Toronto,ON,Islington-city Centre West,43.6461,-79.526,2023-07-28,11,"495,000","499,900",1,1,550,W6674724,ROYAL LEPAGE SIGNATURE REALTY,2023-07-17,9,224,9 Mabelle Avenue Toronto ON
5,910-761,Bay Street,Toronto,ON,Bay Street Corridor,43.6597,-79.385,2023-07-28,14,"803,000","818,000",1,2,750,C6666920,ROYAL LEPAGE CONNECT REALTY,2023-07-14,761,910,761 Bay Street Toronto ON
6,96,Roseheath Avenue,Toronto,ON,East End-danforth,43.6835,-79.318,2023-07-28,9,"1,401,000","999,900",3,3,<NA>,E6676322,ROYAL LEPAGE GOLDEN RIDGE REALTY,2023-07-19,96,-999,96 Roseheath Avenue Toronto ON
7,202-20,Marina Avenue,Toronto,ON,Long Branch,43.5937,-79.5328,2023-07-28,37,"8,200,000","888,888",2,2,1100,W6190872,RIGHT AT HOME REALTY,2023-06-21,20,202,20 Marina Avenue Toronto ON
8,11-38,Gibson Avenue,Toronto,ON,Weston,43.7017,-79.5072,2023-07-28,66,"625,000","629,000",2,2,950,W6029476,REAL ONE REALTY INC.,2023-05-23,38,11,38 Gibson Avenue Toronto ON
9,621-55,Merchant's Wharf,Toronto,ON,Waterfront Communities C8,43.6447,-79.3641,2023-07-28,22,"738,000","759,000",1,1,650,C6641856,RE/MAX PROFESSIONALS INC.,2023-07-06,55,621,55 Merchant's Wharf Toronto ON
